# Hybrid Execution Automatic Switching Demo

In [1]:
import snowflake.snowpark.modin.plugin
import modin.pandas as pd
import numpy as np
import datetime
import pandas as native_pd
from snowflake.snowpark.session import Session; session = Session.builder.create()

Initiating login request with your identity provider. A browser window should have opened for you to complete the login. If you can't see it, check existing browser windows, or your OS settings. Press CTRL+C to abort and try again...
Going to open: https://snowbiz.okta.com/app/snowflake/exk8wfsfryJIn4IWZ2p7/sso/saml?SAMLRequest=jVJdc9owEPwrHvUZSzZuQjRABkKYuA0fDaZMeVNsGVRkydXJMeTXR%2Bajkz4k0zfNafd27%2Fa6t%2FtCei%2FcgNCqhwKfII%2BrVGdCbXpomYxbHeSBZSpjUiveQwcO6LbfBVbIkg4qu1VP%2FE%2FFwXqukQLafPRQZRTVDARQxQoO1KZ0MZg80tAnlAFwY50cOlMyEE5ra21JMa7r2q%2FbvjYbHBJCMLnBDtVAvqB3EuXnGqXRVqdaXih7N9MHEgEmUSPhEE5hfiYOhTqt4DOV5xMI6EOSzFvz2SJB3uAy3Z1WUBXcLLh5ESlfPj2eDIBzsJjOVg%2Bz5eLeB6XrXLIdT3VRVtZ1890L5zzDUm%2BE21E86qFyJzJ4%2BC22hwkLZXudtK%2BHZTRedXbDH98jM%2Blsxeuvm%2Fv59GqVbzaDFHk%2FL4mGTaIxQMVj1eRoXYmEX1skagVREhIatWk78MMrskbeyOUoFLNH5sVsY%2FFZvPp6Z9nRHCtL%2FNc35vtdp84hN4dvsYri1TosrzGAxk1M6HQp9GjA9P93%2Fi5%2Bzzof29TtPx7NtRTpwRtrUzD7cTyBHxwrImvlRyjlBRNykGWGA7iYpNT1neHMupu2puII90%2Bq%2F151%2Fw0%3D&RelaySt

## Example 1: Working with small/inline-created dataframe is faster

In [2]:
us_holidays = [
    ("New Year's Day", "2025-01-01"),
    ("Martin Luther King Jr. Day", "2025-01-20"),
    ("Presidents' Day", "2025-02-17"),
    ("Memorial Day", "2025-05-26"),
    ("Juneteenth National Independence Day", "2025-06-19"),
    ("Independence Day", "2025-07-04"),
    ("Labor Day", "2025-09-01"),
    ("Columbus Day", "2025-10-13"),
    ("Veterans Day", "2025-11-11"),
    ("Thanksgiving Day", "2025-11-27"),
    ("Christmas Day", "2025-12-25")
]

# Create DataFrame
df_us_holidays = pd.DataFrame(us_holidays, columns=["Holiday", "Date"])

# Convert Date column to datetime
df_us_holidays["Date"] = pd.to_datetime(df_us_holidays["Date"])

In [3]:
assert df_us_holidays.get_backend() == 'Pandas'  # with auto, we should expect this to be local

In [4]:
# Add new columns for transformations
df_us_holidays["Day_of_Week"] = df_us_holidays["Date"].dt.day_name()
df_us_holidays["Month"] = df_us_holidays["Date"].dt.month_name()

In [5]:
df_us_holidays

,Holiday,Date,Day_of_Week,Month
0,New Year's Day,2025-01-01,Wednesday,January
1,Martin Luther King Jr. Day,2025-01-20,Monday,January
2,Presidents' Day,2025-02-17,Monday,February
3,Memorial Day,2025-05-26,Monday,May
4,Juneteenth National Independence Day,2025-06-19,Thursday,June
5,Independence Day,2025-07-04,Friday,July
6,Labor Day,2025-09-01,Monday,September
7,Columbus Day,2025-10-13,Monday,October
8,Veterans Day,2025-11-11,Tuesday,November
9,Thanksgiving Day,2025-11-27,Thursday,November


In [6]:
%%time
#Note that without auto-switching, this took 2.5 min
for index, row in df_us_holidays.iterrows():
    print(f"{row['Holiday']} falls on {row['Day_of_Week']}, {row['Month']} {row['Date'].day}, {row['Date'].year}.")

New Year's Day falls on Wednesday, January 1, 2025.
Martin Luther King Jr. Day falls on Monday, January 20, 2025.
Presidents' Day falls on Monday, February 17, 2025.
Memorial Day falls on Monday, May 26, 2025.
Juneteenth National Independence Day falls on Thursday, June 19, 2025.
Independence Day falls on Friday, July 4, 2025.
Labor Day falls on Monday, September 1, 2025.
Columbus Day falls on Monday, October 13, 2025.
Veterans Day falls on Tuesday, November 11, 2025.
Thanksgiving Day falls on Thursday, November 27, 2025.
Christmas Day falls on Thursday, December 25, 2025.
CPU times: user 123 ms, sys: 6.53 ms, total: 129 ms
Wall time: 126 ms


Automatic engine switch happens when merged with large dataset.

In [7]:
df_transactions = pd.read_snowflake("REVENUE_TRANSACTIONS")

Switcheroo: cost to move to pandas: 1000 cost to stay: 250


In [8]:
df_transactions["DATE"] = pd.to_datetime(df_transactions["DATE"])

In [9]:
len(df_us_holidays), len(df_transactions)

(11, 10000000)

In [10]:
combined = pd.merge(df_us_holidays, df_transactions, left_on="Date", right_on="DATE")

BackendCostCalculator Results: Pandas:1000/1000,*Snowflake:500/10000000


Transferring data from Pandas to Snowflake ...:   0%|          | 0/2 [00:00<?, ?it/s]

In [11]:
assert combined.get_backend() == 'Snowflake'

### 💡 Automatic switching speeds up loops/iterations on small data + inline creation of dataframes

## Example 2: When data is filtered the choice of engine changes

Run the following SQL to generate a synthetic dataset with 10M rows of transactions (from 2024-2025 current date)
```sql
CREATE OR REPLACE TABLE revenue_transactions (
    Transaction_ID STRING,
    Date DATE,
    Revenue FLOAT
);

SET num_days = (SELECT DATEDIFF(DAY, '2024-01-01', CURRENT_DATE));
INSERT INTO revenue_transactions (Transaction_ID, Date, Revenue)
SELECT
    UUID_STRING() AS Transaction_ID,
    DATEADD(DAY, UNIFORM(0, $num_days, RANDOM()), '2024-01-01') AS Date,
    UNIFORM(10, 1000, RANDOM()) AS Revenue
FROM TABLE(GENERATOR(ROWCOUNT => 10000000));
```

In [12]:
# Run the following to generate a synthetic dataset with 10M rows of transactions (from 2024-2025 current date)
session.sql('''
CREATE OR REPLACE TABLE revenue_transactions (
    Transaction_ID STRING,
    Date DATE,
    Revenue FLOAT
);''').collect()
session.sql('''SET num_days = (SELECT DATEDIFF(DAY, '2024-01-01', CURRENT_DATE));''').collect()
session.sql('''INSERT INTO revenue_transactions (Transaction_ID, Date, Revenue)
SELECT
    UUID_STRING() AS Transaction_ID,
    DATEADD(DAY, UNIFORM(0, $num_days, RANDOM()), '2024-01-01') AS Date,
    UNIFORM(10, 1000, RANDOM()) AS Revenue
FROM TABLE(GENERATOR(ROWCOUNT => 10000000));
''').collect()

[Row(number of rows inserted=10000000)]

In [13]:
df_transactions = pd.read_snowflake("REVENUE_TRANSACTIONS")

Switcheroo: cost to move to pandas: 1000 cost to stay: 250


In [14]:
len(df_transactions)

10000000

Perform some operations on 10M rows with Snowflake

In [15]:
df_transactions["DATE"] = pd.to_datetime(df_transactions["DATE"])

In [16]:
df_transactions.groupby("DATE").sum()["REVENUE"]

Switcheroo: cost to move to pandas: 1000 cost to stay: 250


DATE
2024-01-01    10781619.0
2024-01-02    10766823.0
2024-01-03    10827368.0
2024-01-04    10765155.0
2024-01-05    10894097.0
                 ...    
2025-04-10    10731430.0
2025-04-11    10708849.0
2025-04-12    10805667.0
2025-04-13    10753431.0
2025-04-14    10739540.0
Freq: None, Name: REVENUE, Length: 470, dtype: float64

In [17]:
assert df_transactions.get_backend() == "Snowflake"

So far everything has been happening in Snowflake, since we are working with the full dataset (10M rows). 
Next, we demonstrate what happens when we filter the data down to a smaller dataset below our 500k threshold for automatic switching. 
We showcase two was of doing the filtering: 
- Method 1: Filtering with pandas (lazy evaluation)
- Method 2: Prefilter with SQL during dataframe creation 

In [18]:
df_transactions_filter1 = df_transactions[(df_transactions["DATE"] >= pd.Timestamp.today().date() - pd.Timedelta('7 days')) & (df_transactions["DATE"] < pd.Timestamp.today().date())]

In [19]:
df_transactions_filter1.get_backend()

'Snowflake'

In [20]:
df_transactions_filter1._query_compiler._modin_frame.ordered_dataframe.row_count_upper_bound

1000000000000000000000000000000000000000000

In this case, since the data is already in Snowflake, it stays in Snowflake even after the filtering.

In [21]:
df_transactions_filter1

,TRANSACTION_ID,DATE,REVENUE
33,48a712ca-3f7c-42f7-a555-16cd8a059180,2025-04-13,322.0
95,9ccbfcd6-0876-470a-a9cb-fb8e5fc77819,2025-04-11,452.0
111,d8f67107-eeea-4891-a6f4-6d2064be59c4,2025-04-13,235.0
115,2469dc83-20f3-42e8-879a-de945f28bad1,2025-04-13,829.0
140,bc3abeef-cd56-455f-ad4d-6b29931b3b06,2025-04-09,526.0
...,...,...,...
9999392,ef5dc0c5-6503-4ada-a012-1a3de05020e3,2025-04-08,449.0
9999756,8acd4ab9-a13a-48cf-a3f5-f614fd068a00,2025-04-12,472.0
9999799,e2ec8d11-f497-4e64-9248-b46ffa0ae349,2025-04-08,195.0
9999911,9d375e0e-6071-43c9-8a05-2d3a96a91745,2025-04-13,860.0


In [22]:
df_transactions_filter1._query_compiler._modin_frame.ordered_dataframe.row_count_upper_bound

149005

In [23]:
df_transactions_filter1

,TRANSACTION_ID,DATE,REVENUE
33,48a712ca-3f7c-42f7-a555-16cd8a059180,2025-04-13,322.0
95,9ccbfcd6-0876-470a-a9cb-fb8e5fc77819,2025-04-11,452.0
111,d8f67107-eeea-4891-a6f4-6d2064be59c4,2025-04-13,235.0
115,2469dc83-20f3-42e8-879a-de945f28bad1,2025-04-13,829.0
140,bc3abeef-cd56-455f-ad4d-6b29931b3b06,2025-04-09,526.0
...,...,...,...
9999392,ef5dc0c5-6503-4ada-a012-1a3de05020e3,2025-04-08,449.0
9999756,8acd4ab9-a13a-48cf-a3f5-f614fd068a00,2025-04-12,472.0
9999799,e2ec8d11-f497-4e64-9248-b46ffa0ae349,2025-04-08,195.0
9999911,9d375e0e-6071-43c9-8a05-2d3a96a91745,2025-04-13,860.0


In [24]:
# Repr should 
# (1) perform the repr
# (2) update count on original data frame
# (3) consider moving
df_transactions_filter1 

,TRANSACTION_ID,DATE,REVENUE
33,48a712ca-3f7c-42f7-a555-16cd8a059180,2025-04-13,322.0
95,9ccbfcd6-0876-470a-a9cb-fb8e5fc77819,2025-04-11,452.0
111,d8f67107-eeea-4891-a6f4-6d2064be59c4,2025-04-13,235.0
115,2469dc83-20f3-42e8-879a-de945f28bad1,2025-04-13,829.0
140,bc3abeef-cd56-455f-ad4d-6b29931b3b06,2025-04-09,526.0
...,...,...,...
9999392,ef5dc0c5-6503-4ada-a012-1a3de05020e3,2025-04-08,449.0
9999756,8acd4ab9-a13a-48cf-a3f5-f614fd068a00,2025-04-12,472.0
9999799,e2ec8d11-f497-4e64-9248-b46ffa0ae349,2025-04-08,195.0
9999911,9d375e0e-6071-43c9-8a05-2d3a96a91745,2025-04-13,860.0


In [25]:
# Do we want to check this eager
# Do we expect that after a filter we check for a switcheroo case?
# can we even do this move if the estimated row size is still the size of data table
assert df_transactions_filter1.get_backend() == "Snowflake" 

In [26]:
print(f"Date range: {df_transactions_filter1['DATE'].min().date()} to {df_transactions_filter1['DATE'].max().date()}. Resulting dataset size: {len(df_transactions_filter1)}")

Date range: 2025-04-07 to 2025-04-13. Resulting dataset size: 149005


Now let's perform filtering via SQL directly, so the dataframe upon creation is small.

In [27]:
df_transactions_filter2 = pd.read_snowflake("SELECT * FROM revenue_transactions WHERE Date >= DATEADD( 'days', -7, current_date ) and Date < current_date")

Switcheroo: cost to move to pandas: 37 cost to stay: 750


Transferring data from Snowflake to Pandas ...:   0%|          | 0/2 [00:00<?, ?it/s]

In [28]:
df_transactions_filter2

,TRANSACTION_ID,DATE,REVENUE
0,23ffd8b3-cc26-4c6c-b697-611d3fb3edd5,2025-04-07,825.0
1,c9f2f434-13b0-4337-b311-95b09e1b115b,2025-04-08,974.0
2,06a3fd2c-c531-4e49-8de3-982ebfee53eb,2025-04-11,281.0
3,03446aff-1f14-41e2-84a2-8201e9ba6ce6,2025-04-10,747.0
4,fee5a29a-7b15-4754-8328-81dd517bf502,2025-04-08,438.0
...,...,...,...
149000,4ec6e5b1-b450-4eb3-b27f-09708837af23,2025-04-09,224.0
149001,b7fe3db2-634c-4cbd-b9bc-f59a9a62eca1,2025-04-12,146.0
149002,14b09669-8f9b-46ca-850d-27e7966a325c,2025-04-13,424.0
149003,c2eeddfd-5d6a-436b-9853-aed04e4416b3,2025-04-07,481.0


In [29]:
# Verify the result is same as above
print(f"Date range: {df_transactions_filter2['DATE'].min()} to {df_transactions_filter2['DATE'].max()}. Resulting dataset size: {len(df_transactions_filter2)}")

Date range: 2025-04-07 to 2025-04-13. Resulting dataset size: 149005


In [30]:
assert df_transactions_filter2.get_backend() == "Pandas" 

In [31]:
len(df_transactions_filter2)

149005

Once you are in pandas, you can still continue to perform the same operations: 

In [32]:
%time
df_transactions_filter1.groupby("DATE").sum()["REVENUE"]

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 6.91 µs
Switcheroo: cost to move to pandas: 37 cost to stay: 750


Transferring data from Snowflake to Pandas ...:   0%|          | 0/2 [00:00<?, ?it/s]

DATE
2025-04-07    10720785.0
2025-04-08    10647216.0
2025-04-09    10769646.0
2025-04-10    10731430.0
2025-04-11    10708849.0
2025-04-12    10805667.0
2025-04-13    10753431.0
Freq: None, Name: REVENUE, dtype: float64

In [33]:
df_transactions_filter1.shape

(149005, 3)

In [34]:
%time
df_transactions_filter2.groupby("DATE").sum()["REVENUE"]

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 5.96 µs


DATE
2025-04-07    10720785.0
2025-04-08    10647216.0
2025-04-09    10769646.0
2025-04-10    10731430.0
2025-04-11    10708849.0
2025-04-12    10805667.0
2025-04-13    10753431.0
Name: REVENUE, dtype: float64

### 💡 Automatic switching means that pandas work well for both small and large data

## Example 3: Performing Custom `apply` on small dataset

Forecast using last year's transaction data via a custom apply function

In [35]:
start_date = pd.Timestamp("2025-10-01")
end_date = pd.Timestamp("2025-10-31")

In [36]:
# Forecasting function using df.apply

def forecast_revenue(df, start_date, end_date):
    # Filter data from last year
    df_filtered = df[(df["DATE"] >= start_date - pd.Timedelta(days=365)) & (df["DATE"] < start_date)]
    
    # Append future dates to daily_avg for prediction
    future_dates = pd.date_range(start=start_date, end=end_date, freq="D")
    df_future = pd.DataFrame({"DATE": future_dates})

    # Group by DATE and calculate the mean revenue
    daily_avg = df_filtered.groupby("DATE")["REVENUE"].mean().reset_index()
    daily_avg["DATE"] = daily_avg["DATE"].astype('datetime64[ns]')
    # Merge future dates with predicted revenue, filling missing values
    df_forecast = df_future.merge(daily_avg, on="DATE", how="left")
    #breakpoint()
    import numpy as np
    # Fill missing predicted revenue with overall mean from last year
    df_forecast["PREDICTED_REVENUE"] = np.nan
    df_forecast["PREDICTED_REVENUE"].fillna(daily_avg["REVENUE"].mean(), inplace=True)
    df_forecast["PREDICTED_REVENUE"] = df_forecast["PREDICTED_REVENUE"].astype("float")
    return df_forecast

In [37]:
df_forecast = forecast_revenue(df_transactions, start_date, end_date)
len(df_forecast)

Switcheroo: cost to move to pandas: 0 cost to stay: 750


Transferring data from Snowflake to Pandas ...:   0%|          | 0/2 [00:00<?, ?it/s]

31

The resulting dataframe is very small, since it is only the 1-month window we're performing forecast on.

In [38]:
assert df_forecast.get_backend() == 'Pandas'

In [39]:
def adjust_for_holiday_weekend(row):
    # For national holidays, revenue down 5% since stores are closed. For weekends, revenue is up 5% due to increased activity.
    if row["DATE"].strftime('%Y-%m-%d') in list(df_us_holidays["Date"].dt.strftime('%Y-%m-%d')): 
        return row["PREDICTED_REVENUE"] * 0.95
    elif row["DATE"].weekday() == 5 or row["DATE"].weekday() == 6: #Saturday/Sundays
        return row["PREDICTED_REVENUE"] * 1.05
    return row["PREDICTED_REVENUE"]

In [40]:
# Adjust for holidays using the apply function
df_forecast["PREDICTED_REVENUE"] = df_forecast.apply(adjust_for_holiday_weekend, axis=1)
df_forecast[["DATE","PREDICTED_REVENUE"]]

,DATE,PREDICTED_REVENUE
0,2025-10-01,505.116721
1,2025-10-02,505.116721
2,2025-10-03,505.116721
3,2025-10-04,530.372557
4,2025-10-05,530.372557
5,2025-10-06,505.116721
6,2025-10-07,505.116721
7,2025-10-08,505.116721
8,2025-10-09,505.116721
9,2025-10-10,505.116721


In [41]:
assert df_forecast.get_backend() == 'Pandas'

In [42]:
print(f"Altair takes in {type(df_forecast)} with {df_forecast.get_backend()} as backend, since we implement the dataframe interchange protocol")

Altair takes in <class 'modin.pandas.dataframe.DataFrame'> with Pandas as backend, since we implement the dataframe interchange protocol


In [43]:
import altair as alt
alt.data_transformers.disable_max_rows()

chart_predicted = alt.Chart(df_forecast).mark_line(color='blue').encode(
    x='monthdate(DATE):T',
    y=alt.Y('PREDICTED_REVENUE:Q',scale=alt.Scale(domain=[470, 550])),
    tooltip=['DATE', 'PREDICTED_REVENUE']
)
chart_predicted

ValueError: invalid literal for int() with base 10: ''

alt.Chart(...)

In [44]:
df_transactions_filtered = df_transactions[
    (df_transactions["DATE"] >= start_date - pd.Timedelta(days=365)) &
    (df_transactions["DATE"] < end_date - pd.Timedelta(days=365))
]
df_transactions_filtered_groupby = df_transactions_filtered.groupby("DATE")["REVENUE"].mean().reset_index()

Switcheroo: cost to move to pandas: 0 cost to stay: 750


Transferring data from Snowflake to Pandas ...:   0%|          | 0/2 [00:00<?, ?it/s]

In [45]:
print(f"Altair takes in {type(df_transactions_filtered_groupby)} with {df_transactions_filtered_groupby.get_backend()} as backend, since we implement the dataframe interchange protocol")

Altair takes in <class 'modin.pandas.dataframe.DataFrame'> with Pandas as backend, since we implement the dataframe interchange protocol


In [46]:
df_forecast_labeled = df_forecast.copy()
df_forecast_labeled['Label'] = 'Predicted Revenue'
df_forecast_labeled = df_forecast_labeled.rename(columns={'PREDICTED_REVENUE': 'Value'})

df_last_year_labeled = df_transactions_filtered_groupby.copy()
df_last_year_labeled['Label'] = 'Revenue'
df_last_year_labeled = df_last_year_labeled.rename(columns={'REVENUE': 'Value'})

# Combine
combined_df = pd.concat([
    df_forecast_labeled[['DATE', 'Value', 'Label']],
    df_last_year_labeled[['DATE', 'Value', 'Label']]
])

# Plot with Value on X and color based on Label
final_chart = alt.Chart(combined_df).mark_line().encode(
    y=alt.Y('Value:Q',scale=alt.Scale(domain=[470, 550])),
    x='monthdate(DATE):T',
    color=alt.Color('Label:N', legend=alt.Legend(title='Type')),
    tooltip=['DATE', 'Value', 'Label']
).properties(
    title='Revenue vs Predicted Revenue (by Value)'
)

final_chart

ValueError: invalid literal for int() with base 10: ''

alt.Chart(...)

### 💡 Apply on small dataset is much faster with automatic switching running with pandas locally.